In [75]:
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels
import hvplot.xarray

In [76]:
station = "Calcofi"

## Load DATA


### CMEMS


In [77]:
data_cmems = xr.open_dataset(f"../data/1_products/{station}_cmems.zarr", engine="zarr").load()
data_cmems

<xarray.Dataset> Size: 3GB
Dimensions:              (time: 9436, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * time                 (time) datetime64[ns] 75kB 1998-01-01 ... 2023-11-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 1GB 26.31...
    npp                  (time, latitude, longitude) float64 428MB 262.0 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 1GB 110.1...
    zooc                 (time, latitude, longitude) float64 428MB 0.1867 ......
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Climato


In [78]:
for i in range(1, 13):
    print(
        f"Month {i} : ",
        set(
            pd.DataFrame({"month": data_cmems.indexes["time"].month, "day": data_cmems.indexes["time"].day})
            .query(f"month == {i}")["day"]
            .unique()
        ),
    )

Month 1 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 2 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29}
Month 3 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 4 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 5 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 6 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 7 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 8 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}


In [79]:
data_cmems_climato = data_cmems.groupby("time.dayofyear").mean()
data_cmems_climato

<xarray.Dataset> Size: 133MB
Dimensions:              (dayofyear: 366, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * dayofyear            (dayofyear) int64 3kB 1 2 3 4 5 ... 362 363 364 365 366
Data variables:
    T                    (dayofyear, depth, latitude, longitude) float64 50MB ...
    npp                  (dayofyear, latitude, longitude) float64 17MB 378.5 ...
    pelagic_layer_depth  (dayofyear, depth, latitude, longitude) float64 50MB ...
    zooc                 (dayofyear, latitude, longitude) float64 17MB 0.184 ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Expand to original time series interval


In [80]:
all_years = []
for year in range(int(data_cmems.time.min().dt.year), int(data_cmems.time.max().dt.year + 1)):
    slice_max = 366
    if not pd.Timestamp(f"{year}-12-31").is_leap_year:
        slice_max = 365

    yearly_data = data_cmems_climato.sel(dayofyear=slice(1, slice_max)).assign_coords(
        time=(
            "dayofyear",
            pd.date_range(
                start=f"{year}-01-01",
                end=f"{year}-12-31",
                freq="D",
                name="time",
            ),
        ),
    )

    all_years.append(yearly_data.swap_dims({"dayofyear": "time"}))
data_cmems_climato_expanded = xr.concat(all_years, dim="time").drop_vars("dayofyear")
data_cmems_climato_expanded.time.attrs = data_cmems.time.attrs
data_cmems_climato_expanded

<xarray.Dataset> Size: 3GB
Dimensions:              (time: 9496, depth: 3, latitude: 55, longitude: 103)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 440B 0.5 1.5 2.5 ... 52.5 53.5 54.5
  * longitude            (longitude) float64 824B -179.5 -178.5 ... -78.5 -77.5
  * time                 (time) datetime64[ns] 76kB 1998-01-01 ... 2023-12-31
Data variables:
    T                    (time, depth, latitude, longitude) float64 1GB 27.84...
    npp                  (time, latitude, longitude) float64 430MB 378.5 ... nan
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 1GB 98.56...
    zooc                 (time, latitude, longitude) float64 430MB 0.184 ... nan
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Plot


In [81]:
data_cmems_climato_expanded.mean(["latitude", "longitude"]).hvplot.line(
    x="time",
    groupby="depth",
    title=f"Climato {station} - CMEMS T",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'a0fc5a5f-b376-433f-b357-b7e2c87a96dd': {'version…

## Export


In [82]:
data_cmems_climato_expanded.to_zarr(
    f"../data/1_products/{station}_cmems_climato.zarr",
    mode="w",
)